In [1]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import json

In [2]:
# Припустимо, у вас є список категорій:
LABEL_COLUMNS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Додаємо новий клас "not_toxic"
LABEL_COLUMNS.append('not_toxic')

# 1. Завантажуємо модель BERT
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(LABEL_COLUMNS))

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# 2. Компіляція моделі
bert_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [4]:
# 3. Завантаження токенізованих даних
# Передбачається, що дані вже містять токенізовані вектори input_ids та attention_mask
# Ваша таблиця повинна мати колонки 'input_ids', 'attention_masks' і мітки класів

data_path = '/kaggle/input/train-data/train_data.csv'
df = pd.read_csv(data_path)

# Конвертація токенізованих даних з рядків у масиви
for column in ['input_ids', 'attention_masks']:
    df[column] = df[column].apply(eval).apply(np.array)

# Виділяємо токенізовані вектори та мітки
input_ids = np.stack(df['input_ids'].values)
attention_mask = np.stack(df['attention_masks'].values)
labels = np.array(df[LABEL_COLUMNS[:-1]].values)  # Виключаємо "not_toxic" для обчислення

# Додаємо колонку "not_toxic"
df['not_toxic'] = (labels.sum(axis=1) == 0).astype(int)
labels = np.hstack([labels, df[['not_toxic']].values])

In [5]:
# 4. Балансування вибірки
balanced_data = []
for label in range(labels.shape[1]):
    subset = df[df[LABEL_COLUMNS[label]] == 1]
    if len(subset) < 1000:
        subset = resample(subset, replace=True, n_samples=1000, random_state=42)
    balanced_data.append(subset)

balanced_df = pd.concat(balanced_data)

# Оновлюємо токенізовані вектори та мітки
input_ids = np.stack(balanced_df['input_ids'].values)
attention_mask = np.stack(balanced_df['attention_masks'].values)
labels = np.array(balanced_df[LABEL_COLUMNS].values)

In [6]:
# 5. Розділення на тренувальну та тестову вибірки
train_input_ids, val_input_ids, train_labels, val_labels = train_test_split(
    input_ids, labels, test_size=0.2, random_state=42
)
train_attention_mask, val_attention_mask = train_test_split(
    attention_mask, test_size=0.2, random_state=42
)

In [7]:
# 6. Тренування моделі
history = bert_model.fit(
    {'input_ids': train_input_ids, 'attention_mask': train_attention_mask},
    train_labels,
    validation_data=(
        {'input_ids': val_input_ids, 'attention_mask': val_attention_mask},
        val_labels
    ),
    epochs=3,
    batch_size=32
)

Epoch 1/3
4475/4475 [==============================] - 2325s 511ms/step - loss: 0.0765 - accuracy: 0.9544 - val_loss: 0.0612 - val_accuracy: 0.9626
Epoch 2/3
4475/4475 [==============================] - 2283s 510ms/step - loss: 0.0493 - accuracy: 0.9700 - val_loss: 0.0572 - val_accuracy: 0.9652
Epoch 3/3
4475/4475 [==============================] - 2283s 510ms/step - loss: 0.0335 - accuracy: 0.9702 - val_loss: 0.0499 - val_accuracy: 0.9537


In [8]:
val_predictions = bert_model.predict({'input_ids': val_input_ids, 'attention_mask': val_attention_mask})
predicted_labels = (val_predictions.logits > 0).astype(int)  # Мультикласова бінаризація

print("Classification Report:")
print(classification_report(val_labels, predicted_labels, target_names=LABEL_COLUMNS))

print("Confusion Matrix:")
conf_matrix = []
for i in range(len(LABEL_COLUMNS)):
    conf_matrix.append(confusion_matrix(val_labels[:, i], predicted_labels[:, i]))
    print(f"Confusion Matrix for {LABEL_COLUMNS[i]}:")
    print(conf_matrix[-1])

1119/1119 [==============================] - 185s 163ms/step
Classification Report:
               precision    recall  f1-score   support

        toxic       0.93      0.91      0.92      6797
 severe_toxic       0.83      0.93      0.88      1309
      obscene       0.93      0.96      0.95      5068
       threat       0.92      0.97      0.94       457
       insult       0.88      0.96      0.92      4909
identity_hate       0.90      0.93      0.91      1092
    not_toxic       0.98      0.99      0.98     28748

    micro avg       0.95      0.97      0.96     48380
    macro avg       0.91      0.95      0.93     48380
 weighted avg       0.95      0.97      0.96     48380
  samples avg       0.96      0.96      0.96     48380

Confusion Matrix:
Confusion Matrix for toxic:
[[28560   437]
 [  609  6188]]
Confusion Matrix for severe_toxic:
[[34245   240]
 [   97  1212]]
Confusion Matrix for obscene:
[[30351   375]
 [  182  4886]]
Confusion Matrix for threat:
[[35297    40]
 [   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
# 7. Створення нової моделі для збереження у форматі HDF5
input_ids_layer = tf.keras.Input(shape=(None,), dtype=tf.int32, name='input_ids')
attention_mask_layer = tf.keras.Input(shape=(None,), dtype=tf.int32, name='attention_mask')

# Отримання виходу з BERT
bert_outputs = bert_model.bert(input_ids=input_ids_layer, attention_mask=attention_mask_layer)
logits = bert_model.classifier(bert_outputs[1])  # Використання "pooled_output"

# Створення нової функціональної моделі
new_model = tf.keras.Model(inputs=[input_ids_layer, attention_mask_layer], outputs=logits)

# Зберігаємо нову модель у форматі HDF5
new_model.save("model-1.h5")

# Збереження історії тренування в JSON-файл
with open('training_history.json', 'w') as json_file:
    json.dump(history.history, json_file)

/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
